In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
import json
import os
import yaml
from dask.distributed import Client

In [4]:
with open('/g/g92/haridev/.dlio_profiler/configuration.yaml', 'r') as file:
    dlp_yaml = yaml.safe_load(file)

In [5]:
# to make this notebook in dev mode.
import sys
sys.path.insert(0,dlp_yaml["APP"])
sys.path

['/usr/WS2/haridev/dlio-profiler',
 '/usr/WS2/haridev/dlio-profiler/examples/dlp_analyzer',
 '/usr/WS2/haridev/dlio-profiler/examples/dlp_analyzer',
 '/usr/workspace/iopp/projects/digio',
 '/collab/usr/gapps/python/build/spack-toss4.1/var/spack/environments/python/._view/75prb56irmif5ejtirjthpx6kq3gqo52/lib/python39.zip',
 '/collab/usr/gapps/python/build/spack-toss4.1/var/spack/environments/python/._view/75prb56irmif5ejtirjthpx6kq3gqo52/lib/python3.9',
 '/collab/usr/gapps/python/build/spack-toss4.1/var/spack/environments/python/._view/75prb56irmif5ejtirjthpx6kq3gqo52/lib/python3.9/lib-dynload',
 '',
 '/usr/WS2/haridev/venvs/venv-dlp_analuzer/lib/python3.9/site-packages']

In [6]:
import dlp_analyzer
print(dlp_analyzer.__file__)
from dlp_analyzer.main import DLPAnalyzer,get_dlp_configuration,update_dlp_configuration,setup_logging,setup_dask_cluster, reset_dask_cluster, get_dlp_configuration

/usr/WS2/haridev/dlio-profiler/dlp_analyzer/__init__.py


In [7]:
dask_run_dir = os.path.join(dlp_yaml["APP"], "dlp_analyzer", "dask", "run_dir")
dask_run_dir

'/usr/WS2/haridev/dlio-profiler/dlp_analyzer/dask/run_dir'

In [8]:
with open (os.path.join(dask_run_dir, "scheduler.json"), "r") as f:
    dask_scheduler = json.load(f)
dask_scheduler

{'type': 'Scheduler',
 'id': 'Scheduler-49282d5e-c1cb-4278-b27b-a8ae5142ab07',
 'address': 'tcp://134.9.71.20:10001',
 'services': {'dashboard': 41547},
 'started': 1698301148.9001098,
 'workers': {}}

In [9]:
conf = update_dlp_configuration(dask_scheduler=dask_scheduler["address"], verbose=True, log_file="./dlp.log", rebuild_index=False, time_approximate=True, host_pattern=r'ruby(\d+)')


In [10]:
conf = get_dlp_configuration()
conf.time_approximate

True

In [11]:
setup_logging()

In [12]:
setup_dask_cluster()

[INFO] [15:48:38] Initialized Client with 48 workers and link http://134.9.71.20:41547/status [/usr/WS2/haridev/dlio-profiler/dlp_analyzer/main.py:563]


In [13]:
reset_dask_cluster()

[INFO] [15:49:58] Restarting all workers [/usr/WS2/haridev/dlio-profiler/dlp_analyzer/main.py:555]


In [14]:
filename = "/p/lustre1/iopp/dlp/cosmoflow-profile-compress-meta-scale-all-512/.trace*.pfw.gz"
#filename = "/p/lustre1/iopp/dlp/cosmoflow-profile-compress-meta-scale-all-512/.trace-3*-of-512.pfw.gz"
filename = "/usr/workspace/iopp/kogiou1/dlio_benchmark_logs/unet3d/node_32_ppn_4/.trace*.pfw.gz"

In [15]:
analyzer = DLPAnalyzer(filename, time_granularity=10e6)

[INFO] [15:50:00] Created index for 128 files [/usr/WS2/haridev/dlio-profiler/dlp_analyzer/main.py:296]
[INFO] [15:50:00] Total size of all files are <dask.bag.core.Item object at 0x1554a9267640> bytes [/usr/WS2/haridev/dlio-profiler/dlp_analyzer/main.py:298]
[INFO] [15:50:00] Loading 768 batches out of 128 files and has 11996460 lines overall [/usr/WS2/haridev/dlio-profiler/dlp_analyzer/main.py:311]
[INFO] [15:50:12] Loaded events [/usr/WS2/haridev/dlio-profiler/dlp_analyzer/main.py:342]


In [16]:
analyzer.events["hostname"].unique().compute()[:2]

0    ruby165
1    ruby926
Name: hostname, dtype: string

In [17]:
items = analyzer.summary()
items

[INFO] [15:50:12] Total number of events in the workload are 11996204 [/usr/WS2/haridev/dlio-profiler/dlp_analyzer/main.py:432]
/usr/WS2/haridev/venvs/venv-dlp_analuzer/lib/python3.9/site-packages/dask/dataframe/core.py:5983: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=(None, 'int64'))

  warnings.warn(meta_warning(meta))
/usr/WS2/haridev/venvs/venv-dlp_analuzer/lib/python3.9/site-packages/dask/dataframe/core.py:5983: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please pr

╭──────────────────────────────────────────────────── Summary ────────────────────────────────────────────────────╮
│  Allocation    Scheduler Allocation Details                                                                     │
│                ├── Nodes: 32 ruby[165,1253]                                                                     │
│                ├── Processes: 128 [0,127]                                                                       │
│                ├── Thread allocations across nodes (includes dynamically created threads)                       │
│                │   ├── Compute: 128                                                                             │
│                │   └── I/O: 2304                                                                                │
│                └── Events Recorded: 11996204                                                                    │
│  Dataset       Description of Dataset Used                                                                      │
│                └── Files: 171 ['/p/lustreX/kogiouX/dlio_benchmark_results/unetXd/valid',                        │
│                    '/p/lustreX/kogiouX/dlio_benchmark_results/unetXd/train/img_XXX_of_XXX.npz', '<NA>',         │
│                    '/p/lustreX/kogiouX/dlio_benchmark_results/unetXd/train']                                    │
│  I/O Behavior  Behavior of Application                                                                          │
│                ├── Split of Time in application                                                                 │
│                │   ├── Compute: 6.838 sec                                                                       │
│                │   ├── Overall I/O: 54.486 sec                                                                  │
│                │   ├── Unoverlapped I/O: 47.649 sec                                                             │
│                │   └── Unoverlapped Compute: 0.000 sec                                                          │
│                ├── Transfer size distribution by function                                                       │
│                │   ├── Function       |min   |25    |mean  |median|75    |max   |                               │
│                │   ├── opendir        |NA    |nan   |nan   |NA    |nan   |NA    |                               │
│                │   ├── __xstat64      |NA    |nan   |nan   |NA    |nan   |NA    |                               │
│                │   ├── open64         |NA    |nan   |nan   |NA    |nan   |NA    |                               │
│                │   ├── __fxstat64     |NA    |nan   |nan   |NA    |nan   |NA    |                               │
│                │   ├── lseek64        |NA    |nan   |nan   |NA    |nan   |NA    |                               │
│                │   ├── read           |NA    |4MB   |4MB   |4MB   |4MB   |4MB   |                               │
│                │   └── close          |NA    |nan   |nan   |NA    |nan   |NA    |                               │
│                └── Event count by function                                                                      │
│                    ├── opendir : 5372                                                                           │
│                    ├── __xstat64 : 5372                                                                         │
│                    ├── open64 : 17822                                                                           │
│                    ├── __fxstat64 : 35614                                                                       │
│                    ├── lseek64 : 10442002                                                                       │
│                    ├── read : 732133                                                                            │
│                    └── close : 15288                  